In [1]:
#Initialise the spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DatasetToCompleteTheSixSparkExercises') .getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import *

In [3]:
df_1 = spark.read.parquet('hdfs:///R07/sales_parquet/*.parquet', infetSchema=True, header= True)
# df = spark.read.parquet('DatasetToCompleteTheSixSparkExercises/products_parquet', inferSchema=True, header=True)
df_2 = spark.read.parquet('hdfs:///R07/products_parquet/*.parquet', infetSchema=True, header= True)
df_3 = spark.read.parquet('hdfs:///R07/sellers_parquet/*.parquet', infetSchema=True, header= True)

In [4]:
# df_1.show(5)
df_3.show(5)

+---------+-----------+------------+
|seller_id|seller_name|daily_target|
+---------+-----------+------------+
|        0|   seller_0|     2500000|
|        1|   seller_1|      257237|
|        2|   seller_2|      754188|
|        3|   seller_3|      310462|
|        4|   seller_4|     1532808|
+---------+-----------+------------+
only showing top 5 rows



In [5]:
df_1.groupBy(df_1.date).agg(F.countDistinct(df_1.product_id))

DataFrame[date: string, count(product_id): bigint]

## Question 1. How many distinct products are sold each day?

In [81]:
product_count = df_1.groupBy(df_1.date).agg(F.countDistinct(df_1.product_id).alias("no of distinct products sold")).orderBy('date')
product_count.show()


+----------+----------------------------+
|      date|no of distinct products sold|
+----------+----------------------------+
|2020-07-01|                      100337|
|2020-07-02|                       99807|
|2020-07-03|                      100017|
|2020-07-04|                       99791|
|2020-07-05|                       99796|
|2020-07-06|                      100765|
|2020-07-07|                       99756|
|2020-07-08|                       99662|
|2020-07-09|                      100501|
|2020-07-10|                       98973|
+----------+----------------------------+



### Question 2 What is the average daily revenue of each product?

In [82]:
df2_df1_join = df_1.join(df_2, df_1["product_id"] == df_2["product_id"], "inner")
a = df2_df1_join.agg(avg(df_1['num_pieces_sold'] * df_2['price']).alias('average revenue'))
a.show()

+------------------+
|   average revenue|
+------------------+
|1246.1338560822878|
+------------------+



### Question 3. What is the average daily revenue of each product?


In [86]:
df2_df1_avg = df2_df1_join.groupBy(df_1["product_id"], df_1["date"]).agg(avg(df_1['num_pieces_sold'] * df_2['price']).alias("revenue"))

In [97]:
df2_df1_avg.groupBy('product_id').agg(avg('Revenue').alias("daily average revenue")).show()
# df2_df1_avg.groupBy('product_id').agg(avg('Revenue')).filter(col('product_id')== '0').show()

+----------+---------------------+
|product_id|daily average revenue|
+----------+---------------------+
|  10005243|               4312.0|
|  10023464|               1121.0|
|  10050363|               1764.0|
|  10089524|               5300.0|
|  10122266|               2225.0|
|  10134574|               3864.0|
|  10150439|               5112.0|
|  10158822|               3010.0|
|  10160884|               4628.0|
|  10172594|               4860.0|
|  10175294|               3248.0|
|   1017716|               5412.0|
|  10200802|               1722.0|
|  10215353|               1232.0|
|  10218345|              13818.0|
|  10220977|              10032.0|
|  10255853|               3120.0|
|  10288525|               8316.0|
|  10304712|               1152.0|
|  10324080|                650.0|
+----------+---------------------+
only showing top 20 rows



### Question 4. For each seller, what is the average % contribution of an order to the sellers daily quota?

In [14]:
df_seller_sales = df_1.join(df_3, df_1["seller_id"] == df_3["seller_id"], "inner" ).withColumn(
    "ratio", df_1["num_pieces_sold"]/df_3["daily_target"])

In [91]:
df_seller_sales.groupBy(df_3["seller_id"]).agg(avg("ratio").alias("average contribution(%)")).orderBy('seller_id').show()

+---------+-----------------------+
|seller_id|average contribution(%)|
+---------+-----------------------+
|        0|   2.019885898946922...|
|        1|   1.964233366461015E-4|
|        2|   6.690408001060484E-5|
|        3|   1.628885370565939...|
|        4|   3.296428039825817E-5|
|        5|   4.211073965904023E-5|
|        6|   4.782147194369122E-5|
|        7|   2.595228787788170...|
|        8|   9.213030375408863E-5|
|        9|   3.837913136180238E-5|
+---------+-----------------------+



### Question 5. Who are the second most selling and the least selling persons (sellers) for each product?

In [92]:
# Join seller & product tables 
df_sellers_join = df_1.join(df_3, df_3['seller_id'] == df_1['seller_id'], 'inner').groupBy('product_id', df_3['seller_id']).agg(sum('num_pieces_sold').alias('num_pieces_sold'))\
.orderBy('product_id', 'num_pieces_sold')

In [17]:
df_sellers_join.show()

+----------+---------+---------------+
|product_id|seller_id|num_pieces_sold|
+----------+---------+---------------+
|         0|        0|   9.59445802E8|
|  10000006|        4|           39.0|
|  10000047|        9|           29.0|
|  10000271|        5|           51.0|
|   1000030|        5|           89.0|
|   1000042|        9|           47.0|
|  10000429|        6|            6.0|
|  10000601|        8|            6.0|
|  10000607|        2|           30.0|
|   1000065|        7|           33.0|
|  10000712|        7|           59.0|
|  10000715|        6|           51.0|
|  10000865|        1|           67.0|
|  10000971|        9|           50.0|
|   1000101|        5|           61.0|
|  10001082|        1|            4.0|
|  10001097|        3|           92.0|
|  10001174|        9|           71.0|
|  10001234|        5|           68.0|
|   1000124|        9|           73.0|
+----------+---------+---------------+
only showing top 20 rows



In [93]:
#Seller Rank added
df_product_rank  = Window.partitionBy("product_id").orderBy(col("num_pieces_sold").desc())

df_sellers_join = df_sellers_join.withColumn("sellerRank", row_number().over(df_product_rank))\
.orderBy(col("sellerRank").desc())

df_sellers_join.show()

+----------+---------+---------------+----------+
|product_id|seller_id|num_pieces_sold|sellerRank|
+----------+---------+---------------+----------+
|  32602520|        9|           12.0|         3|
|   3534470|        3|           25.0|         3|
|  20774718|        9|            5.0|         3|
|  40496308|        5|           52.0|         3|
|  28592106|        5|           42.0|         3|
|  34681047|        5|           19.0|         3|
|  52606213|        7|           28.0|         3|
|  14542470|        5|            3.0|         3|
|  36269838|        8|           16.0|         3|
|  67723231|        5|           26.0|         3|
|  72017876|        1|           20.0|         3|
|  35669461|        4|           39.0|         3|
|  18182299|        7|           15.0|         3|
|  69790381|        5|            8.0|         3|
|  10978356|        7|           27.0|         3|
|  31136332|        9|           57.0|         3|
|  17944574|        8|           15.0|         3|


In [94]:
df_product_max_rank  = Window.partitionBy("product_id")

df_sellers_join = df_sellers_join.withColumn("worstRank", max(col("sellerRank")).over(df_product_max_rank))\
.orderBy(col("worstRank").desc())

df_sellers_join.show()

+----------+---------+---------------+----------+---------+
|product_id|seller_id|num_pieces_sold|sellerRank|worstRank|
+----------+---------+---------------+----------+---------+
|  36269838|        8|           16.0|         3|        3|
|  61475460|        9|           88.0|         1|        3|
|  40496308|        8|           68.0|         1|        3|
|  14542470|        9|           35.0|         2|        3|
|  61475460|        7|           41.0|         3|        3|
|  19986717|        2|           53.0|         1|        3|
|  36269838|        1|           38.0|         2|        3|
|  31136332|        8|           99.0|         1|        3|
|  17944574|        2|           32.0|         1|        3|
|  14542470|        2|           62.0|         1|        3|
|  61475460|        6|           64.0|         2|        3|
|  19986717|        9|           18.0|         2|        3|
|  19986717|        1|            9.0|         3|        3|
|  40496308|        9|           58.0|  

In [95]:
def second_most_selling(sellerRank, worstRank, seller_id):
    if worstRank == 1:
        return seller_id
    elif sellerRank == 2:
        return seller_id  
    
def least_selling(sellerRank, worstRank, seller_id):
    if worstRank == 1:
        return seller_id
    elif sellerRank == worstRank:
        return seller_id

second = udf(second_most_selling, StringType())
least = udf(least_selling, StringType())

df_sellers_join = df_sellers_join.withColumn('second_most_selling', second('sellerRank', 'worstRank', 'seller_id'))
df_sellers_join = df_sellers_join.withColumn('least_selling', least('sellerRank', 'worstRank', 'seller_id'))

df_sellers_join.show()

+----------+---------+---------------+----------+---------+-------------------+-------------+
|product_id|seller_id|num_pieces_sold|sellerRank|worstRank|second_most_selling|least_selling|
+----------+---------+---------------+----------+---------+-------------------+-------------+
|  31136332|        8|           99.0|         1|        3|               null|         null|
|  31136332|        9|           57.0|         3|        3|               null|            9|
|  31136332|        1|           85.0|         2|        3|                  1|         null|
|  17944574|        2|           32.0|         1|        3|               null|         null|
|  17944574|        8|           15.0|         3|        3|               null|            8|
|  17944574|        5|           17.0|         2|        3|                  5|         null|
|  14542470|        2|           62.0|         1|        3|               null|         null|
|  14542470|        5|            3.0|         3|        3| 

In [96]:
df_sellers_join = df_sellers_join.groupBy('product_id')\
.agg(max('second_most_selling').alias('second_top_sell'),\
     max('least_selling').alias('least_sell'),\
     count('seller_id').alias('no_of_sellers'))\
.orderBy(col('no_of_sellers').desc())

df_sellers_join.show()

+----------+---------------+----------+-------------+
|product_id|second_top_sell|least_sell|no_of_sellers|
+----------+---------------+----------+-------------+
|  32602520|              2|         9|            3|
|   3534470|              9|         3|            3|
|  20774718|              3|         9|            3|
|  40496308|              9|         5|            3|
|  28592106|              4|         5|            3|
|  34681047|              6|         5|            3|
|  52606213|              9|         7|            3|
|  14542470|              9|         5|            3|
|  36269838|              1|         8|            3|
|  67723231|              6|         5|            3|
|  72017876|              4|         1|            3|
|  35669461|              3|         4|            3|
|  18182299|              4|         7|            3|
|  69790381|              3|         5|            3|
|  10978356|              9|         7|            3|
|  31136332|              1|